In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import cv2
import csv
import os
import time
import math
from sortedcontainers import SortedKeyList, SortedDict
from PIL import Image
import statistics

In [ ]:
# MOB subtractor
subtractor = cv2.createBackgroundSubtractorMOG2(detectShadows=False)

In [ ]:
# kp and des methods

sift = cv2.xfeatures2d.SIFT_create()
def siftKpAndDes(frame):
    return sift.detectAndCompute(frame,None)

surf = cv2.xfeatures2d.SURF_create()
def surfKpAndDes(frame):
    return surf.detectAndCompute(frame,None)

star = cv2.xfeatures2d.StarDetector_create()
def fastKp(frame):
    return star.detect(frame,None)

brief = cv2.xfeatures2d.BriefDescriptorExtractor_create()
def briefDes(frame, kps):
    return brief.compute(frame, kps)

orb = cv2.ORB_create()
def orbKpAndDes(frame):
    return orb.detectAndCompute(frame,None)

In [ ]:
# matcher methods

FLANN_INDEX_KDTREE = 1
FLANN_KDTREES = 5

FLANN_INDEX_LSH = 6
TABLE_NUMBER = 6 #12 6
KEY_SIZE = 12 #20 12
MULTI_PROBE_LEVEL = 1 #2 1
SEARCH_CHECKS = 50

#BF matching
bf_orb = cv2.BFMatcher(cv2.NORM_HAMMING)
bf_sift = cv2.BFMatcher()
def bfMatcher(des1, des2, feature_type = 'SIFT', ratio = 0.7):

    if feature_type is 'ORB': 
        matches = bf_orb.knnMatch(des1,des2,k=2)
    else:
        matches = bf_sift.knnMatch(des1, des2, k=2)
    # filter using ratio
    good = []
    for pair in matches:
        try:
            m,n = pair
            if m.distance < ratio*n.distance:
                good.append(m)
        except ValueError:
            print('Missing Match pair!')
            pass
    return good, matches


#FLANN-based
search_params = dict(checks = SEARCH_CHECKS)
flann_orb = cv2.FlannBasedMatcher(dict(algorithm = FLANN_INDEX_LSH,
                   table_number = TABLE_NUMBER,
                   key_size = KEY_SIZE,
                   multi_probe_level = MULTI_PROBE_LEVEL),
                   search_params)
flann_sift = cv2.FlannBasedMatcher(dict(algorithm = FLANN_INDEX_KDTREE, trees = FLANN_KDTREES), 
                                   search_params )
def flannMatcher(des1, des2, feature_type = 'SIFT', ratio=0.7):
    
    if feature_type is 'ORB': 
        matches = flann_orb.knnMatch(des1, des2, k=2)
    else:
        matches = flann_sift.knnMatch(des1, des2, k=2)
    # filter using ratio
    good = []
    for pair in matches:
        try:
            m,n = pair
            if m.distance < ratio*n.distance:
                good.append(m)
        except ValueError:
            print('Missing Match pair!')
            pass
    return good, matches
        

In [ ]:
# intersection of two segments given their endpoints
# This code is contributed by Ansh Riyal 

# Hash, String, and comparison definitions provided by Eric Zeng
class Point: 
    def __init__(self, x, y): 
        self.x = x 
        self.y = y
        self.key = x * 1000 + y * 1000
        
    def __eq__(self, other):
        return (self.x == other.x) and (self.y == other.y)
        
    def __lt__(self, other):
        return self.x < other.x or (self.x == other.x and self.y < other.y)
    
    def __str__(self):
        return "Point(" + str(self.x).zfill(5) + ", " + str(self.y).zfill(5) + ")"

    def __hash__(self):
        return hash(str(self.x) + ',' + str(self.y))
        
  
# Given three colinear points p, q, r, the function checks if  
# point q lies on line segment 'pr'  
def onSegment(p, q, r): 
    if ( (q.x <= max(p.x, r.x)) and (q.x >= min(p.x, r.x)) and 
           (q.y <= max(p.y, r.y)) and (q.y >= min(p.y, r.y))): 
        return True
    return False
  
def orientation(p, q, r): 
    # to find the orientation of an ordered triplet (p,q,r) 
    # function returns the following values: 
    # 0 : Colinear points 
    # 1 : Clockwise points 
    # 2 : Counterclockwise 
      
    # See https://www.geeksforgeeks.org/orientation-3-ordered-points/amp/  
    # for details of below formula.  
      
    val = (float(q.y - p.y) * (r.x - q.x)) - (float(q.x - p.x) * (r.y - q.y)) 
    if (val > 0): 
          
        # Clockwise orientation 
        return 1
    elif (val < 0): 
          
        # Counterclockwise orientation 
        return 2
    else: 
          
        # Colinear orientation 
        return 0
  
# the main function that returns true if  
# the line segment 'p1q1' and 'p2q2' intersect. 
def doIntersect(p1,q1,p2,q2): 
      
    # Find the 4 orientations required for  
    # the general and special cases 
    o1 = orientation(p1, q1, p2) 
    o2 = orientation(p1, q1, q2) 
    o3 = orientation(p2, q2, p1) 
    o4 = orientation(p2, q2, q1) 
  
    # General case 
    if ((o1 != o2) and (o3 != o4)): 
        return True
  
    # Special Cases 
  
    # p1 , q1 and p2 are colinear and p2 lies on segment p1q1 
    if ((o1 == 0) and onSegment(p1, p2, q1)): 
        return True
  
    # p1 , q1 and q2 are colinear and q2 lies on segment p1q1 
    if ((o2 == 0) and onSegment(p1, q2, q1)): 
        return True
  
    # p2 , q2 and p1 are colinear and p1 lies on segment p2q2 
    if ((o3 == 0) and onSegment(p2, p1, q2)): 
        return True
  
    # p2 , q2 and q1 are colinear and q1 lies on segment p2q2 
    if ((o4 == 0) and onSegment(p2, q1, q2)): 
        return True
  
    # If none of the cases 
    return False


In [ ]:
# Line class by Eric Zeng
class Line:
    def __init__(self, p, q):
        self.p = p
        self.q = q
        self.key = str(p.y).zfill(5) + str(q.y).zfill(5) # lines should be sorted by the y-coors
    
    def __eq__(self, other):
        return (self.p is other.p and self.q is other.q)
    
    def __lt__(self, other):
        return self.p < other.p
    
    def __str__(self):
        return "Line[" + str(self.p).zfill(5) + "-" + str(self.q).zfill(5) + "]"
    
    def __hash__(self):
        return hash(str(self.p) + '-' + str(self.q))
    
# Evaluates if two lines share any endpoints    
def endsCoincident(p1, p2):
#     print("COINCIDENT EP TEST")
#     print(p1.p)
#     print(p1.q)
#     print(p2.p)
#     print(p2.q)
#     print(p1.p == p2.p)
#     print(p1.p == p2.q)
#     print(p1.q == p2.p)
#     print(p1.q == p2.q)
    return (p1.p == p2.p) or (p1.p == p2.q) or (p1.q == p2.p) or (p1.q == p2.q)

# assumes pts are 4 points creating a closed quad
# electing to use list instead of BST or other varients since n=4 (finite, known, and very small)
# This implementation was written by Eric Zeng
def checkSimplePolygon(pts):
    print('INTERSECTION CHECK')
#     print(pts)

    # create 4 segments
    # sort the endpoints in each segment to have the smaller x value be first
    segs = [
            sorted([Point(int(pts[0][0][0] + 10000), int(pts[0][0][1] + 10000)), 
                    Point(int(pts[1][0][0] + 10000), int(pts[1][0][1] + 10000))]), 
            sorted([Point(int(pts[1][0][0] + 10000), int(pts[1][0][1] + 10000)), 
                    Point(int(pts[2][0][0] + 10000), int(pts[2][0][1] + 10000))]),
            sorted([Point(int(pts[2][0][0] + 10000), int(pts[2][0][1] + 10000)), 
                    Point(int(pts[3][0][0] + 10000), int(pts[3][0][1] + 10000))]),
            sorted([Point(int(pts[3][0][0] + 10000), int(pts[3][0][1] + 10000)), 
                    Point(int(pts[0][0][0] + 10000), int(pts[0][0][1] + 10000))])
           ]
    # sort all segments against each other using the starting endpoint's x value
    segs = sorted(segs, key = lambda x: x[0])
#     [print(str(s[0]) + ', ' + str(s[1])) for s in segs]
    segs = [Line(s[0], s[1]) for s in segs] # covert sublists to lines
    
    # init event queue, marking {"Adds" and "Exit" endpoints -> Line} mappings
    eventq = SortedDict()
    for s in segs:
        left = str(s.p) + 'A'
        right = str(s.q) + 'E'
        if (left not in eventq):
            eventq.update({left: list()})
        if (right not in eventq):
            eventq.update({right: list()})
            
        eventq.get(left).append(s)
        eventq.get(right).append(s)
    
#     print("EVENT QUEUE ORDER: ")
#     [print(a) for a in eventq]

    # init sweep line
    sl = SortedKeyList([], key=lambda x: x.key)
    for e in eventq:
#         print(e)
#         print('targets: ' + ",".join(str(line) for line in eventq[e]))
#         print('SL: ' + ",".join(str(line) for line in sl))
        if e[-1] is 'A':
#             print('ADDS')
            for line in eventq[e]:
                sl.add(line)
#                 print(line)
                try: 
#                     print('above')
                    above = sl[sl.index(line) + 1]
                    if (not endsCoincident(above, line)) and doIntersect(above.p, above.q, line.p, line.q):
#                         print('above FALSE')
                        return False
                except IndexError:
#                     print('above index error')
                    pass # missing above
                try: 
#                     print('below')
                    below = sl[sl.index(line) - 1]
                    if (not endsCoincident(below, line)) and doIntersect(below.p, below.q, line.p, line.q):
#                         print('below FALSE')
                        return False
                except IndexError:
#                     print('below index error')
                    pass # missing above
        else: # e[0] is 'E'
#             print('REMOVES')
            for line in eventq[e]:
                # check above and below intersections if not conincident
#                 print(line)
                try:
#                     print('both')
                    above = sl[sl.index(line) + 1]
                    below = sl[sl.index(line) - 1]
                    if (not endsCoincident(above, below)) and doIntersect(above.p, above.q, below.p, below.q):
#                         print('both FALSE')
                        return False
                except IndexError:
#                     print('both index error')
                    pass # missing either/both line above or below in SL, no need for additional check
                 
                # once check is done, discard this line from the SL
                
                sl.remove(line)

#         [print(l) for l in sl]
#         print()
    return True

# testpoly = [[[30, 30]], [[56, 56]], [[52, 32]], [[32, 52]] ]

# test = cv2.polylines(img1,[np.int32(testpoly)],True,(0,255,0),3, cv2.LINE_AA)  
# plt.figure(figsize=(20,20))
# plt.imshow(test)

# checkSimplePolygon(testpoly)

In [ ]:
# python3 program to evaluate 
# area of a polygon using 
# shoelace formula, clockwise in opencv coordinate system
# 
# NOTE: assumes polygon is simple and closed (non-intersecting)
# please check before simple property before running
# (X[i], Y[i]) are coordinates of i'th point. 
# This code is contributed by Smitha Dinesh Semwal 
# https://www.geeksforgeeks.org/area-of-a-polygon-with-given-n-ordered-vertices/
def shoelace(points): 
    
    #transform list of Point object to two lists of X and Y coordinates
    X = []
    Y = []
    for p in points:
        X.append(p.x)
        Y.append(p.y)
  
    # Initialze area 
    area = 0.0
  
    # Calculate value of shoelace formula 
    j = len(points) - 1
    for i in range(0,len(points)): 
        area += (X[j] + X[i]) * (Y[j] - Y[i]) 
        j = i   # j is previous vertex to i 
      
  
    # Return absolute value 
    return abs(area / 2.0)

In [ ]:
# Simple Polygon Area comparison utilizing shoelace above
def checkArea(pts, total_area, min_area):
    print('AREA CHECK')
    points = [
        Point(pts[0][0][0], pts[0][0][1]),
        Point(pts[1][0][0], pts[1][0][1]),
        Point(pts[2][0][0], pts[2][0][1]),
        Point(pts[3][0][0], pts[3][0][1]),
    ]
    area = shoelace(points)
    ratio = area / total_area
    return (ratio < 1.0 and ratio > min_area), ratio
    

In [ ]:
# Interior angle measures function
# assumes quadilateral is non-intersecting! Check before running or function outpus garbage
#
# Algorithm from:
# https://math.stackexchange.com/questions/149959/how-to-find-the-interior-angle-of-an-irregular-pentagon-or-polygon
#
# Implementation by Eric Zeng
def measureAngles(pts, min_angle, max_angle):
    print('ANGLE CHECK')
    points = [
        Point(pts[0][0][0], pts[0][0][1]),
        Point(pts[1][0][0], pts[1][0][1]),
        Point(pts[2][0][0], pts[2][0][1]),
        Point(pts[3][0][0], pts[3][0][1]),
    ]
    
    angles = []
    
    #start top left, go CW in opencv coordinate space
    for i in range(4):
        past = points[i-1]
        curr = points[i]
        nxt = points[i+1] if i < 3 else points[0]
        vec1 = Point(curr.x - past.x, curr.y - past.y)
        vec2 = Point(nxt.x - curr.x, nxt.y - curr.y)
        vec1m = [vec1.x, vec1.y, 0]
        vec2m = [vec2.x, vec2.y, 0]
        cp = np.cross(vec1m, vec2m) # cross product for direction check 

        value = np.dot(vec1m, vec2m)/(math.sqrt(vec1.x**2 + vec1.y**2)*math.sqrt(vec2.x**2 + vec2.y**2))
        value = -1 * value if cp[2] < 0 else value
        angles.append(math.pi - math.acos(value))
    
    angles = [x*180/math.pi for x in angles] # to degrees
    if (sum(angles) > 180): # slightly above 2*pi for floating point inaccuracies
        angles = [x - math.pi for x in angles]
    return (not any([x > max_angle or x < min_angle for x in angles])), angles

In [ ]:
# Point in Polygon (Contour) filter
# Used to deterine which keypoints are within the bounding box of a homography transformation; True if inside
# Implementation by Eric Zeng
def filterPoints(pts, h_pts):
    contour = np.array([[p[0][0], p[0][1]] for p in h_pts])
    ret = []
    for point in pts:
        ret.append(False if cv2.pointPolygonTest(contour, (point[0], point[1]), False) < 0 else True)
    return ret

# cv2.drawContours(img1, [pts], 0, (255,255,255), 1)
# print(cv2.pointPolygonTest(pts, (10,10), False))

In [ ]:
# Container class to represent a positive match to be saved in our CSV output
# Author: Eric Zeng
class PositiveMatch:
    def __init__(self, img, name, area, smallest_angle, largest_angle, cwf):
        self.img = img
        self.name = name
        self.area = area
        self.smallest_angle = smallest_angle
        self.largest_angle = largest_angle
        self.cwf = cwf

In [ ]:
# Script to output results to CSV
# Author: Eric Zeng
def outputCSV(positives, errors, query, video, output_dir, chain, feature_ext, matcher,
               frame_interval, ratio, min_matches, fps,
                min_area, allow_intersections, min_angle, max_angle):
    print("Analysis finished!")
        
    areas = []
    min_angles = []
    max_angles = []
    # TOTO: consider saving dict of positives alongside csv file to allow for futher analytics
    # TODO: write out detailed Errors too
    with open(os.path.join(output_dir,'results.csv'), 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['SUMMARY OF PARAMS'])
        writer.writerow(['Query Image', 'Video', 'Chain Length', 'Feature Extractor', 'Matcher', 
                         'Frame Interval', 'Pass Ratio', 'Match Thresh',
                        'Min Area', 'Filter Intersections', 'Min Angle', 'Max Angle'])
        writer.writerow([query, video, chain, feature_ext, matcher, frame_interval, ratio, min_matches,
                        min_area, not allow_intersections, min_angle, max_angle])
        writer.writerow(['MATCHES'])
        writer.writerow(['Frame Num', 'Img Name', 'Area','Min Angle', 'Max Angle', 'Chain Weight Factor'])
        for match in positives.items():
            writer.writerow([match[0], match[1].name, match[1].area, match[1].smallest_angle, match[1].largest_angle,
                             match[1].cwf])
            areas.append(match[1].area)
            min_angles.append(match[1].smallest_angle)
            max_angles.append(match[1].largest_angle)
            # TODO: more metadata
        mean_area = statistics.mean(areas)
        mean_min_angle = statistics.mean(min_angles)
        mean_max_angle = statistics.mean(max_angles)
        mean_mean_angle = statistics.mean([mean_min_angle, mean_max_angle])
        stdev_area = statistics.stdev(areas)
        stdev_min_angle = statistics.stdev(min_angles)
        stdev_max_angle = statistics.stdev(max_angles)
        mean_stdev_angle = statistics.mean([stdev_min_angle, stdev_max_angle])
        writer.writerow([len(positives), len(positives) * frame_interval/fps,
                         mean_area, mean_min_angle, mean_max_angle, mean_mean_angle]) 
                        #TODO: write sum/avg/count/std dev statistics here
        writer.writerow(['','',stdev_area, stdev_min_angle, stdev_max_angle, mean_stdev_angle])
        writer.writerow(['ERRORS'])
        for err in errors:
            writer.writerow([err])
            
# Adds on-screen display information to the target img
def addOSD(tgt, n_pos, chain_n, frame_n, delta):
    cv2.putText(tgt, "T.Count: " + str(n_pos), (2,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    cv2.putText(tgt, "Pos.Chain: " + str(chain_n), (2,60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    cv2.putText(tgt, "Frame.Num: " + str(frame_n) + delta, (2, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

In [ ]:
# preprocess kp/des for query image(s)

img1 = cv2.imread('logos/allianz.jpg',0) # queryImage
# find the keypoints and descriptors
kp1, des1 = siftKpAndDes(img1)

img1kps = cv2.drawKeypoints(img1, kp1, None, color=(0,255,0), flags=0)
plt.figure(figsize=(20,20))
plt.imshow(img1kps), plt.show()

print(len(kp1))

In [ ]:
# encoding params for quality when saving sample jpeg screenshots
encoding_params = [int(cv2.IMWRITE_JPEG_QUALITY), 10]

def runMatching(query, video, output_dir="outputs", chain = 3, feature_ext = 'ORB', matcher = 'FLANN',
               frame_interval = 1, ratio = 0.7, min_matches = 15, fps = 30,
                min_area='0.1', allow_intersections = False, min_angle = 45, max_angle = 135,
               multiple = True):
    positives = dict() # dict of frame numbers of positive indentifications
    n_pos = 0 # total count of positives
    errors = [] # list of error frames to analyze later (hopefully should be empty!)
    
    cap = cv2.VideoCapture(video)
    img_chain = [] # list of past images in the current running chain of positive frames, up to the chain parameter
    
    # preprocess kp/des for query image(s)
    img1 = cv2.imread(query, 0)
    if feature_ext is 'ORB':     
        kp1, des1 = orbKpAndDes(img1)
    elif feature_ext is 'SIFT':
        kp1, des1 = siftKpAndDes(img1)
    else:
        # no features or matching
        kp1, des1 = [], []
    
    
    # frame number tracker
    frame_n = 0
    
    # start time for running speed measure
    start_t = time.process_time()
    delta = ''
    
    while(True):
        for i in range(frame_interval - 1):
            #skip i number of frames
            cap.grab()

        # Capture frame-by-frame and covert to grayscale
        ret, frame = cap.read()
        if not ret:
            break # exit when done
            
        frame_n += 1 # ret is True, frame found, continue processing
        
        # running speed display (delta is % speed of realtime according to input fps and frame interval)
        if frame_n % fps is 0:
            end_t = time.process_time()
            delta = int((100.0 * frame_interval) / (end_t - start_t))
            delta = " (" + str(delta) + "%)"
            print(delta)
            start_t = time.process_time()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#         # TESTING: MOG2 background subtractor mask
#         mask = subtractor.apply(gray, masked, 0.999)
#         masked = cv2.bitwise_and(gray, masked)
#         kp2, des2 = siftKpAndDes(masked)

#         # TESTING: Equalize historgram to incrase contrast (probably for ORB)
#         eh = cv2.equalizeHist(gray)
#         kp2, des2 = siftKpAndDes(eh)

#         # TESTING: Blurring with various LPFs
#         blurred = cv2.bilateralFilter(gray,9,75,75)
#         kp2, des2 = siftKpAndDes(blurred)

        # DEFAULT: find the keypoints and descriptors
        if feature_ext is 'ORB':     
            kp2, des2 = orbKpAndDes(gray)
        elif feature_ext is 'SIFT':
            kp2, des2 = siftKpAndDes(gray)
        else:
            # no features or matching
            cv2.putText(gray, "Frame.Num: " + str(frame_n) + delta, (2, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
            cv2.imshow('frame', gray)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            continue
            
        # kp/des generation done, we can covert back to color so homography lines can be shown
        img3 = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)

        add_to_chain = False # flag: if we should record this video frame to the img_chain
        sum_area = 0 # total % coverage by all valid homography matches
        allAngles = [] # list of all angles in all valid homography matches
        
        # loop and keeping finding valid matches w/ homography until no more in this video frame
        while True:

            # list of good matches (need to init to empty for debug mode)
            good = []

            # check flags
            areaCheck = False
            intersectionCheck = False
            angleCheck = False
            try:
                if matcher is 'FLANN':
                    good, matches = flannMatcher(des1, des2, feature_type=feature_ext, ratio=ratio)
                elif matcher is 'BF':
                    good, matches = bfMatcher(des1, des2, feature_type=feature_ext, ratio=ratio)
                else:
                    # DEBUGGING: show only target image kps, no matching run
                    img3 = cv2.drawKeypoints(img3, kp2, None, color=(0,255,0), flags=0)
                    cv2.putText(img3, "Frame.Num: " + str(frame_n) + delta, (2, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
                    cv2.imshow('frame',img3)
                    break # break out of this video frame's inner loop

                if len(good) > min_matches:
                    print(("Frame%d: " % frame_n) + str(len(good)) + " good matches Found")
                    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
                    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
                    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)

                    if M is None:
                        # if homography matrix is empty
                        print("Empty homography matrix")
#                         img3 = gray # display image without any lines
                        break
                    else:
    #                     print(M)
                        matchesMask = mask.ravel().tolist()
                        h,w = img1.shape
                        pts = np.float32([ [0,0],[w-1,0],[w-1,h-1],[0,h-1] ]).reshape(-1,1,2) # cw direction
                        dst = cv2.perspectiveTransform(pts,M)

                        # TODO: analyze shpae of dst, checking it is reasonable
                        flat = [coor for pt in dst for c in pt for coor in c] # mark as bad if coordinates are extreme (abs >= 10,000)
                        if not any([coor >= 10000 or coor <= -10000 for coor in flat]):
                            total_area = gray.shape[0] * gray.shape[1]
                            areaCheck, homography_area = checkArea(dst, total_area, min_area) if min_area > 0 else (True, 0) # auto pass if min_area is 0
                            if areaCheck: # only bother checking intersections if area is OK
                                intersectionCheck = checkSimplePolygon(dst) if not allow_intersections else True
                                if intersectionCheck: # only bother checking angles if no intersections
                                    angleCheck, angles = measureAngles(dst, min_angle, max_angle) if (min_area > 0 and max_angle < 180) else (True, [-1])
                                    
                        if (not areaCheck):
                            img3 = cv2.polylines(img3,[np.int32(dst)],True,(0,255,255),3, cv2.LINE_AA)
                            break
                        elif (not intersectionCheck):
                            img3 = cv2.polylines(img3,[np.int32(dst)],True,(0,0,255),3, cv2.LINE_AA)
                            break
                        elif (not angleCheck):
                            img = cv2.polylines(img3,[np.int32(dst)],True,(255,0,0),3, cv2.LINE_AA)
                            break
                        else:
                            print('PASSED ALL HOMOGRAPHY CHECKS')
                            img3 = cv2.polylines(img3,[np.int32(dst)],True,(0,255,0),3, cv2.LINE_AA)
                            sum_area += homography_area
                            allAngles = allAngles + angles
                            add_to_chain = True # add to img_chain if we have >= 1 valid homography match
                            
                            if multiple:
                                # remove des2 descriptors that are within this homography transformation
                                # so new homogrpahies can be matched
                                contour = np.array([[int(p[0][0]), int(p[0][1])] for p in dst])
                                kp_xy = np.float32([kp2[idx].pt for idx in range(0, len(kp2))])
                                res = filterPoints(kp_xy, dst)
                                kp2 = np.array([k for i,k in enumerate(kp2) if not res[i]])
                                des2 = np.array([descriptor for i,descriptor in enumerate(des2) if not res[i]])
                            else:
                                break
     
                else:
                    print(("Frame%d: " % frame_n) + "Not enough good matches - {}/{}".format(len(good), min_matches) )
                    matchesMask = None
                    break
#                     img3 = gray # display image without any lines

    #             draw_params = dict(matchColor = (0,255,0), # draw matches in green color
    #                                singlePointColor = None,
    #                                matchesMask = matchesMask, # draw only inliers
    #                                flags = 2)
    #             img3 = cv2.drawMatches(img1,kp1,img2,kp2,good,None,**draw_params)

    #             # Display the resulting frame
    #             cv2.imshow('frame',gray)

            except cv2.error as e:
                print(("Frame%d: " % frame_n) + 'Matching Error! Recording and skipping this frame...')
#                 img3 = gray # display image without any lines
                print(e)
                errors.append(e)
                break

        #  saving samples positives images
#         if len(good) > min_matches and M is not None and areaCheck and intersectionCheck and angleCheck:
        if add_to_chain:
            img_name = 'frame%d.jpg' % frame_n
            pm = PositiveMatch(img3, img_name, sum_area, min(allAngles), max(allAngles), 1)
            img_chain.append(pm) # add this image to the current continuous chain of positive frames
            if len(img_chain) is chain:
                for i,img_in_chain in enumerate(img_chain):
                    n_pos += 1
                    addOSD(img3, n_pos, len(img_chain), frame_n, delta) if i is chain - 1 else None
                    cv2.imwrite(os.path.join(output_dir, img_in_chain.name), img_in_chain.img, encoding_params) # write with compression param as specified up top
                    positives[frame_n - chain + 1 + i] = img_in_chain
                    print('RECORDED: ' + str(img_in_chain.name))
            elif len(img_chain) > chain:
                n_pos += 1
                addOSD(img3, n_pos, len(img_chain), frame_n, delta)
                cv2.imwrite(os.path.join(output_dir, img_name), img3, encoding_params) # write with compression param as specified up top
                positives[frame_n] = pm
                print('RECORDED: ' + str(img_name))
                
                # TODO: save only the single median image in each group of 5?
#             if len(img_chain) % chain is 0:

            else:
                addOSD(img3, n_pos, len(img_chain), frame_n, delta)
        
        else:
            img_chain = [] # if negative, then clear chain
            addOSD(img3, n_pos, len(img_chain), frame_n, delta)
        
        try:
#             cv2.imshow('frame',img3)
            pass
        except:
            print('IMG converstion failed, skipping...')

        if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        elif cv2.waitKey(1) & 0xFF == ord('p'):
            while True:
                if cv2.waitKey(1) & 0xFF == ord('o'):
                     break

                            
               
                    

    # When everything done, release the capture
    cap.release()
    
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    
    # write outputs
    outputCSV(positives, errors, query, video, output_dir, chain, feature_ext, matcher,
               frame_interval, ratio, min_matches, fps,
                min_area, allow_intersections, min_angle, max_angle)
    
    return positives, n_pos, errors

In [ ]:
# runner
# TODO: make all params specified here

FRAME_INTERVAL = 2 # 1 means no frame skips
MIN_MATCH_COUNT =  10 # RULE: >10% of keypoints in query image? -> work well only fort scale-invarient (SIFT)
RATIO = 0.7 # lowered from 0.7 to derease sensitivity
MIN_AREA = 0.01 # 0 to disable area checking

runMatching('logos/rolex.jpg', 'videos/15brazil-5min.mp4', output_dir='outputs', chain=3,
            feature_ext = 'ORB', matcher='FLANN',
            frame_interval=FRAME_INTERVAL, ratio=RATIO, min_matches=MIN_MATCH_COUNT, fps=50, 
            min_area=MIN_AREA,  allow_intersections=False, min_angle=45, max_angle= 135,
           multiple=True)
